In [2]:
import tarfile
import pandas as pd

In [3]:
df = pd.read_csv('NELA2017.tar.bz2', compression='bz2', header=0, sep=' ', quotechar='"', error_bad_lines=False, nrows=1000)

b'Skipping line 241: expected 43471 fields, saw 47561\n'
b'Skipping line 397: expected 43471 fields, saw 83576\nSkipping line 403: expected 43471 fields, saw 123849\n'
b'Skipping line 415: expected 43471 fields, saw 48379\n'
b'Skipping line 493: expected 43471 fields, saw 73892\n'
b'Skipping line 511: expected 43471 fields, saw 48972\n'
C:\Users\alexander.paskal\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (7,10,11,12,13,14,15,16,17,18,19,20,35,36,37,45,46,56,98,103,109,116,117,118,125,128,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,243,244,245,246,247,248,249,250,251,252,2

In [9]:
file = tarfile.open("NELA2017.tar.bz2")

In [13]:
f = tarfile.open("NELA2017.tar.bz2")
f.extractall(path="data1")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Columns: 43471 entries, ./ to agency.txt
dtypes: float64(3822), object(39649)
memory usage: 331.7+ MB


In [8]:
df.head()

,./,Unnamed: 1,Unnamed: 2,Unnamed: 3,13476306163,012400,5,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,aposwill.1,restore.1,trustapos.1,with.112,Kremlin.1,Papadopoulos.16,told.15,Russian.24,news.8,agency.txt
0,20.0,ctime=1559840764,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20.0,atime=1559856357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,23.0,SCHILY.dev=16777224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,24.0,SCHILY.ino=169929488,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,18.0,SCHILY.nlink=1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


./
Unnamed: 1
Unnamed: 2
Unnamed: 3
13476306163
012400
5
Unnamed: 7
Unnamed: 8
Unnamed: 9
Unnamed: 10
Unnamed: 11
13161231765
013663
5.1
Unnamed: 15
Unnamed: 16
Unnamed: 17
Unnamed: 18
Unnamed: 19
13161227364
013343
5.2
Unnamed: 23
Unnamed: 24
Unnamed: 25
Unnamed: 26
Unnamed: 27
13161235753
013520
5.3
Unnamed: 31
Unnamed: 32
Unnamed: 33
Unnamed: 34
Unnamed: 35
13161234775
013546
5.4
Unnamed: 39
Unnamed: 40
Unnamed: 41
Unnamed: 42
Unnamed: 43
13476266606
014103
5.5
Unnamed: 47
Unnamed: 48
Unnamed: 49
Unnamed: 50
Unnamed: 51
13476263015
014551
5.6
Unnamed: 55
Unnamed: 56
Unnamed: 57
Unnamed: 58
Unnamed: 59
13476243452
014224
5.7
Unnamed: 63
Unnamed: 64
Unnamed: 65
Unnamed: 66
Unnamed: 67
13476230076
015247
5.8
Unnamed: 71
Unnamed: 72
Unnamed: 73
Unnamed: 74
Unnamed: 75
13476230407
015246
5.9
Unnamed: 79
Unnamed: 80
Unnamed: 81
Unnamed: 82
Unnamed: 83
13476230751
015251
5.10
Unnamed: 87
Unnamed: 88
Unnamed: 89
Unnamed: 90
Unnamed: 91
13476231331
015245
5.11
Unnamed: 95
Unnamed: 96
Unnamed